In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [3]:
from langchain_community.graphs import Neo4jGraph

In [4]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph

In [5]:
movie_query = """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
m.title = row.title,
m.imdbRating = toFloat(row.imdbRating)

FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))

FOREACH (actor in split(row.actors, '|') |
    MERGE (p: Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))

FOREACH (genre in split(row.genres, '|') |
    MERGE (g: Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))

"""

graph.query(movie_query)

[]

In [6]:
graph.refresh_schema()
print(graph.schema)

Node properties are the following:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT},Person {name: STRING},Genre {name: STRING}
Relationship properties are the following:

The relationships are the following:
(:Movie)-[:IN_GENRE]->(:Genre),(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)


In [7]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [8]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=groq_api_key, model_name="gemma2-9b-it")

In [9]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)

In [10]:
chain.graph_schema

'Node properties are the following:\nMovie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT},Person {name: STRING},Genre {name: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Movie)-[:IN_GENRE]->(:Genre),(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)'

In [46]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]


In [49]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

example_prompt = PromptTemplate.from_template(
    "User input: {question} \n Cypher query: {query}"
)

prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a Neo4J expert. Given an input question, create a syntactically accurate cypher query.",
    suffix="User input: {question} \n Cypher query: ",
    input_variables=["question","schema"]
)

print(prompt.format(question="How many movies has Tom Hanks acted in?",schema="foo"))


You are a Neo4J expert. Given an input question, create a syntactically accurate cypher query.

User input: How many artists are there? 
 Cypher query: MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)

User input: Which actors played in the movie Casino? 
 Cypher query: MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name

User input: How many movies has Tom Hanks acted in? 
 Cypher query: MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)

User input: List all the genres of the movie Schindler's List 
 Cypher query: MATCH (m:Movie {title: 'Schindler\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name

User input: Which actors have worked in movies from both the comedy and action genres? 
 Cypher query: MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name

User input: How many movies has Tom Hanks acted in? 


In [50]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020F847C5310>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020F848F2D10>, model_name='gemma2-9b-it', groq_api_key=SecretStr('**********'))

In [51]:
chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, cypher_prompt=prompt, verbose=True)

In [53]:
chain.invoke("Which actors acted in the movie Casino?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a:Person) RETURN a.name

Full Context:
[{'a.name': 'Robert De Niro'}, {'a.name': 'Joe Pesci'}, {'a.name': 'Sharon Stone'}, {'a.name': 'James Woods'}]

> Finished chain.


{'query': 'Which actors acted in the movie Casino?',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods acted in the movie Casino. \n'}

In [55]:
chain.invoke("Actors who acted in Forrest Gump?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Forrest Gump'})<-[:ACTED_IN]-(a) RETURN a.name 

Full Context:
[]

> Finished chain.


{'query': 'Actors who acted in Forrest Gump?',
 'result': "I don't know the answer. \n"}

In [57]:
chain.invoke("Total number of actors?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Person) RETURN count(a)

Full Context:
[{'count(a)': 1239}]

> Finished chain.


{'query': 'Total number of actors?', 'result': 'There are 1239 actors. \n'}